## Proyecto Final: Control de Gastos 
### Realizado por: David Esteban Solarte Eraso
### Link Video: https://youtu.be/J0_EYulHiF8

In [ ]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import json
import os

# Cargar transacciones
def cargarTransacciones():
    if os.path.exists("transacciones.json"):
        with open("transacciones.json", "r") as archivo:
            return json.load(archivo)
    else:
        return []

# Guardar transacciones
def guardarTransacciones():
    with open("transacciones.json", "w") as archivo:
        json.dump(transacciones, archivo, indent=4)

# Variables globales
transacciones = cargarTransacciones()
categorias = ["Alimentación", "Transporte", "Entretenimiento", "Salud", "Educación", "Servicios", "Otros"]

# Función para registrar ingreso o gasto
def registrar_transaccion(tipo, monto, descripcion, categoria, fecha):
    try:
        monto = float(monto)
        if not categoria:
            return "❌ Selecciona una categoría", None
        
        if not fecha:
            fecha = datetime.now().strftime("%d-%m-%Y")
        
        transaccion = {
            "tipo": tipo,
            "monto": monto,
            "descripcion": descripcion,
            "categoria": categoria,
            "fecha": fecha
        }
        transacciones.append(transaccion)
        guardarTransacciones()
        return "✅ Transacción registrada con éxito", pd.DataFrame(transacciones)
    except ValueError:
        return "❌ Monto inválido", None

# Función para ver el historial
def ver_historial():
    if not transacciones:
        return "Sin transacciones registradas", None, None

    df = pd.DataFrame(transacciones)
    df['fecha_dt'] = pd.to_datetime(df['fecha'], format="%d-%m-%Y", errors='coerce')
    df['mes_nombre'] = df['fecha_dt'].dt.strftime('%B-%Y')
    
    # Gráfico de gastos por categoría
    df_gastos = df[df['tipo'] == 'gasto']
    if df_gastos.empty:
        return "No hay gastos para mostrar", df, None

    resumen = df_gastos.groupby('categoria')['monto'].sum()
    fig, ax = plt.subplots()
    ax.pie(resumen, labels=resumen.index, autopct='%1.1f%%', startangle=140)
    ax.set_title("Distribución de gastos por categoría")
    plt.tight_layout()
    grafico_path = "grafico.png"
    plt.savefig(grafico_path)
    plt.close(fig)

    return "Historial cargado", df, grafico_path

# Función para ver el balance
def ver_balance():
    if not transacciones:
        return "Sin transacciones registradas", None

    df = pd.DataFrame(transacciones)
    ingresos = df[df['tipo'] == 'ingreso']['monto'].sum()
    egresos = df[df['tipo'] == 'gasto']['monto'].sum()
    balance = ingresos - egresos

    fig, ax = plt.subplots()
    ax.bar(["Ingresos", "Gastos"], [ingresos, egresos], color=["green", "red"])
    ax.set_title("Ingresos vs Egresos")
    ax.set_ylabel("Monto ($)")
    plt.tight_layout()
    grafico_path = "balance.png"
    plt.savefig(grafico_path)
    plt.close(fig)

    mensaje = f"💰 Ingresos: ${ingresos:.2f} | 💸 Gastos: ${egresos:.2f} | 🔎 Balance: ${balance:.2f}"
    return mensaje, grafico_path

In [ ]:
with gr.Blocks() as interfaz:
    gr.Markdown("# 💼 Control de Gastos Personales")

    with gr.Tab("➕ Registrar Transacción"):
        tipo = gr.Radio(["ingreso", "gasto"], label="Tipo de transacción")
        monto = gr.Textbox(label="Monto ($)", placeholder="Ej: 150.00")
        descripcion = gr.Textbox(label="Descripción")
        categoria = gr.Dropdown(choices=categorias, label="Categoría")
        fecha = gr.Textbox(label="Fecha (dd-mm-yyyy) [opcional]", placeholder="Dejar en blanco para usar la fecha actual")
        boton_registrar = gr.Button("Guardar")
        salida_registro = gr.Textbox(label="Estado")
        tabla_transacciones = gr.Dataframe()

        boton_registrar.click(fn=registrar_transaccion,
                              inputs=[tipo, monto, descripcion, categoria, fecha],
                              outputs=[salida_registro, tabla_transacciones])

    with gr.Tab("📋 Historial"):
        boton_historial = gr.Button("Ver Historial")
        mensaje_historial = gr.Textbox(label="Mensaje")
        tabla_historial = gr.Dataframe()
        grafico_historial = gr.Image()

        boton_historial.click(fn=ver_historial,
                              inputs=[],
                              outputs=[mensaje_historial, tabla_historial, grafico_historial])

    with gr.Tab("📊 Balance"):
        boton_balance = gr.Button("Ver Balance")
        texto_balance = gr.Textbox(label="Resumen")
        grafico_balance = gr.Image()

        boton_balance.click(fn=ver_balance,
                            inputs=[],
                            outputs=[texto_balance, grafico_balance])

interfaz.launch()